# exp001
最頻値などのナイーブな予測

In [12]:
import os
import sys
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing

import pandas as pd
import itertools

In [13]:
EXP_NAME = "exp001"

In [14]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

os.makedirs(os.path.join(OUTPUT_DIR, EXP_NAME), exist_ok=True)

In [15]:
# データ読み込み
train_sessions = pd.read_pickle(PREP_DIR + "train_sessions.pkl")
test_sessions = pd.read_pickle(PREP_DIR + "test_sessions.pkl")
sub = pd.read_csv(INPUT_DIR + "sample_submission.csv", usecols=["session_type"])

test_sessions["session"] = test_sessions["session"].astype(str)
test_sessions["aid"] = test_sessions["aid"].astype(str)

In [16]:
sessions = test_sessions["session"].unique()

In [17]:
# sessionごとの最頻値
session_aid_cnt = test_sessions.groupby(["session", "aid"]).agg(cnt=("ts", "count")).reset_index()
session_aid_cnt = session_aid_cnt.sort_values(["session", "cnt"], ascending=[True, False])
session_recom = session_aid_cnt.groupby("session").head(20)
session_recom.head()

,session,aid,cnt
0,12899779,59625,1
1,12899780,1142000,2
2,12899780,582732,1
3,12899780,736515,1
4,12899780,973453,1


In [18]:
# 全体の最頻値
total_aid_cnt = test_sessions.groupby("aid").agg(cnt=("ts", "count")).reset_index()
total_aid_cnt = total_aid_cnt.sort_values("cnt", ascending=False)
total_aid_cnt = total_aid_cnt.head(20)
most_freq_aid = total_aid_cnt["aid"].tolist()
total_recom = pd.DataFrame(list(itertools.product(sessions, most_freq_aid)), columns=["session", "aid"])
total_recom.head()

,session,aid
0,12899779,485256
1,12899779,1460571
2,12899779,108125
3,12899779,986164
4,12899779,554660


In [19]:
# sessionごとと全体の結合(sessionを優先)
total_recom["cnt"] = 0
recom = pd.concat([session_recom, total_recom])
recom = recom.sort_values(["session", "cnt"], ascending=[True, False])
recom = recom.drop_duplicates(subset=["session", "aid"])
recom = recom.groupby("session").head(20)
recom = recom.groupby("session")["aid"].apply(list).reset_index()
recom["labels"] = recom["aid"].apply(lambda x: ' '.join(x))
recom.head()

,session,aid,labels
0,12899779,"[59625, 485256, 1460571, 108125, 986164, 55466...",59625 485256 1460571 108125 986164 554660 1551...
1,12899780,"[1142000, 582732, 736515, 973453, 485256, 1460...",1142000 582732 736515 973453 485256 1460571 10...
2,12899781,"[199008, 141736, 194067, 57315, 918667, 485256...",199008 141736 194067 57315 918667 485256 14605...
3,12899782,"[779477, 1494780, 476063, 595994, 834354, 4139...",779477 1494780 476063 595994 834354 413962 562...
4,12899783,"[255297, 300127, 1114789, 1216820, 1729553, 17...",255297 300127 1114789 1216820 1729553 1754419 ...


In [20]:
# sub作成
sub["session"] = sub["session_type"].apply(lambda s: s.split("_")[0])
sub = sub.merge(recom[["session", "labels"]], on="session", how="left")
sub = sub[["session_type", "labels"]].copy()
sub.to_csv(os.path.join(OUTPUT_DIR, EXP_NAME, f"{EXP_NAME}_sub.csv"), index=False)
sub.head(20)

,session_type,labels
0,12899779_clicks,59625 485256 1460571 108125 986164 554660 1551...
1,12899779_carts,59625 485256 1460571 108125 986164 554660 1551...
2,12899779_orders,59625 485256 1460571 108125 986164 554660 1551...
3,12899780_clicks,1142000 582732 736515 973453 485256 1460571 10...
4,12899780_carts,1142000 582732 736515 973453 485256 1460571 10...
5,12899780_orders,1142000 582732 736515 973453 485256 1460571 10...
6,12899781_clicks,199008 141736 194067 57315 918667 485256 14605...
7,12899781_carts,199008 141736 194067 57315 918667 485256 14605...
8,12899781_orders,199008 141736 194067 57315 918667 485256 14605...
9,12899782_clicks,779477 1494780 476063 595994 834354 413962 562...
